In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_distances

In [2]:
# Cargar el dataset
df = pd.read_csv('DATASET\Movies_EDA_ILB.csv', encoding='utf-8')

In [3]:
model_df=df[['overview','title','genres_name']]

In [4]:
def convert_columns_to_lowercase(df):
    for column in df.select_dtypes(include='object'):
        df[column] = df[column].str.lower()
    return df

In [5]:
model_df = convert_columns_to_lowercase(model_df)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\2484958803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.lower()


In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Alarcon
[nltk_data]     Ilbert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Alarcon
[nltk_data]     Ilbert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def create_text(model_data):
    text = model_data['title']
    for column in ['genres_name', 'overview', 'namescast', 'directors']:
        if pd.notnull(model_data[column]) and isinstance(model_data[column], str):
            text += ' ' + model_data[column].lower()
    return text

In [8]:
stop_words = set(stopwords.words('english'))

def extract_keywords(overview):
    if pd.notnull(overview) and isinstance(overview, str):
        tokens = word_tokenize(overview.lower())  # Tokenizar y convertir a minúsculas
        keywords = [token for token in tokens if token.isalpha() and token not in stop_words]  # Filtrar palabras clave
        return keywords
    else:
        return []

# Iterar sobre el dataset y encontrar las palabras clave para cada fila
model_df['keywords'] = None  # Crear la columna 'keywords' inicialmente con valores nulos
for index, row in model_df.iterrows():
    plot = row['overview']
    keywords = extract_keywords(plot)
    
    model_df.at[index, 'keywords'] = keywords

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\682870792.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['keywords'] = None  # Crear la columna 'keywords' inicialmente con valores nulos


In [9]:
model_df.head(3)

,overview,title,genres_name,keywords
0,"led by woody, andys toys live happily in his r...",toy story,"animation, comedy, family","[led, woody, andys, toys, live, happily, room,..."
1,when siblings judy and peter discover an encha...,jumanji,"adventure, fantasy, family","[siblings, judy, peter, discover, enchanted, b..."
2,a family wedding reignites the ancient feud be...,grumpier old men,"romance, comedy","[family, wedding, reignites, ancient, feud, ne..."


In [10]:
model_df.drop(columns=['overview'],inplace=True)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\1078362905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(columns=['overview'],inplace=True)


In [11]:
def remove_commas(text):
    if isinstance(text, str):
        return text.replace(',', '')
    else:
        return ''

In [12]:
def clean_keywords(keywords):
    if isinstance(keywords, list):
        cleaned_keywords = ' '.join(keywords)
        cleaned_keywords = cleaned_keywords.replace('[', '').replace(']', '')
        return cleaned_keywords
    else:
        return ''

# Aplicar la función clean_keywords a la columna 'keywords'
model_df['keywords'] = model_df['keywords'].apply(clean_keywords)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\814766163.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['keywords'] = model_df['keywords'].apply(clean_keywords)


In [13]:
model_df['texto'] = (
    model_df['title'].apply(remove_commas) + ' ' +
    model_df['genres_name'].apply(remove_commas) + ' ' +
    model_df['keywords'].apply(remove_commas) + ' '
)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\3928622450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['texto'] = (


In [14]:
model_df.head(3)

,title,genres_name,keywords,texto
0,toy story,"animation, comedy, family",led woody andys toys live happily room andys b...,toy story animation comedy family led woody an...
1,jumanji,"adventure, fantasy, family",siblings judy peter discover enchanted board g...,jumanji adventure fantasy family siblings judy...
2,grumpier old men,"romance, comedy",family wedding reignites ancient feud neighbor...,grumpier old men romance comedy family wedding...


In [15]:
model_df.drop(columns=['keywords','genres_name'], inplace=True)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11568\3057418384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(columns=['keywords','genres_name'], inplace=True)


In [16]:
model_df=model_df.reset_index(drop=True)

In [17]:
model_df.head(3)

,title,texto
0,toy story,toy story animation comedy family led woody an...
1,jumanji,jumanji adventure fantasy family siblings judy...
2,grumpier old men,grumpier old men romance comedy family wedding...


In [18]:
# Reemplazar los valores np.nan con una cadena vacía
model_df['texto'] = model_df['texto'].fillna('')

In [19]:
model_df.to_csv('DATASET\Movies_ML_ILB.csv',index=False)

In [20]:
# Crear una instancia del CountVectorizer con los parámetros deseados
cv = CountVectorizer(stop_words='english', max_features=100000000000000, min_df=1, max_df=20)  # Ajusta el valor de max_features según tus necesidades

In [21]:
# Definir el tamaño de lote para el cálculo de similitud
batch_size = 7000

In [22]:
# Obtener el número total de muestras y calcular el número de lotes
n_samples = len(model_df)
n_batches = (n_samples // batch_size) + 1

cosine_sim_list = []

for i in range(n_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, n_samples)

    # Obtener el subconjunto de datos
    subset_data = model_df.iloc[start:end]

    # Ajustar y transformar el CountVectorizer en el subconjunto de datos
    cv_matrix = cv.fit_transform(subset_data['texto'])

    # Calcular la similitud coseno entre los textos
    similarity_matrix = cosine_similarity(cv_matrix, cv_matrix)

    # Agregar la matriz de similitud a la lista
    cosine_sim_list.append(similarity_matrix)

In [23]:
# Encontrar la dimensión máxima a lo largo del eje 1
max_dim_1 = max(matrix.shape[1] for matrix in cosine_sim_list)

In [24]:
# Ajustar las dimensiones de las matrices al número máximo de columnas
cosine_sim_list = [np.pad(matrix, ((0, 0), (0, max_dim_1 - matrix.shape[1])), mode='constant') for matrix in cosine_sim_list]

In [25]:
# Combinar las matrices de similitud coseno en una única matriz
cosine_sim = np.concatenate(cosine_sim_list, axis=0)

In [26]:
def recomendaciones(titulo, cosine_sim=cosine_sim):
    try:
        # Getting the index of the movie that matches the title
        idx = model_df[model_df['title'] == str(titulo).lower()].index[0]
        # Getting the similarity scores
        sim_scores = list(enumerate(cosine_sim[idx]))
        #Sorting the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Getting the top 5 recommendations
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]
        recommendations=list(model_df['title'].iloc[movie_indices].str.title())
        return {'lista recomendada': recommendations} 
    except:
        return {'lista recomendada': ['LEGO DC Super Hero Girls: Brain Drain', 'The Lion of Thebes', 'Criminal Justice', 'Mad at the Moon', 'The Far Pavilions']}

In [27]:
recomendaciones('the matrix')

{'lista recomendada': ['Commando',
  'The Matrix Reloaded',
  'The Transformers: The Movie',
  'The Matrix Revolutions',
  'Looker']}